# 1. Iterables and iterators

## What are iterables?


- Any object that you can loop over with a for loop
- Any object that can return its members one at a time
- For instance: lists, tuples, sets, dictionaries, strings, files, generators, etc.

In [ ]:
x = ['Anna', 28]
for item in x:
    print(item)

## What happens in a for loop?

In a for loop, the `iter()` function is called on the iterable object, and then the `next` function is called on the iterator object until a `StopIteration` exception is raised.

In [ ]:
x = ['Anna', 28]
x_iterator = iter(x)
while True:
    try:
        print(next(x_iterator))
    except StopIteration:
        break

Let's look at this process step by step:

- Each iterable object has a "double underscore" method or dunder methos called `__iter__()` that **returns an iterator object**. 
- Alternatively, we can use the `iter()` function. It will call the `__iter__()` method of the list.

In [ ]:
x_iterator = x.__iter__() 
x_iterator = iter(x)        # Same as above, but more common
type(x_iterator)

- Each **iterator object** has a dunder method called `__next__()` that **returns the next item in the sequence.**
- Alternatively, we can use the `next()` function.
- If we reach the end of the sequence, it raises a `StopIteration` exception.
- Since we don't want to run into this exception, the for loop automatically catches it and stops the iteration.

In [ ]:
x = ['Anna', 28]
x_iterator = iter(x)
print(next(x_iterator))
print(next(x_iterator))
print(next(x_iterator))

# 2. Generators

## Motivation

**What happens if the iterable is huge?**, e.g. a list of 1 billion numbers

- It will take up a lot of memory, or maybe not even fit in memory
- It will take a long time to generate the list

**Generators are a helpful alternative to lists in this case.**

- Instead of storing all items in memory, generators produce items one at a time.
- Generators only compute values "on the fly" when needed (_lazy evaluation_).

In [26]:
def square_function(numbers):
    result = []
    for number in numbers:
        result.append(number**2)
    return result

In [61]:
def square_generator(numbers):
    for number in numbers:
        yield number**2

- `square_function` first creates a list of squares and then **returns** the list. If the list is huge, it will take long to generate the list and consume a lot of memory.
- `square_generator` is a generator function. It **yields** the square of each number one at a time. It doesn't store the squares in memory. It only computes the next square when it is needed. If we need the next square, we call `next()` on the generator object.

In [ ]:
results = square_function(range(10))
results

In [ ]:
my_generator = square_generator(range(10))
print(next(my_generator))
print(next(my_generator))

## Generator functions

- Generator functions are functions that use the `yield` keyword, typically inside a loop
- Multiple `yield` statements are allowed
- Generator functions are therefore suitable for more complex scenarios compared to generator expressions

In [ ]:
def transform(numbers):
    for number in numbers:
        yield number+1

    for number in numbers:
        yield number*100

my_generator = transform([1,2,3])
for item in my_generator:
    print(item)

## Generator expressions

- The syntax of generator expressions are similar to list comprehensions, but with parentheses instead of square brackets
- Generator expressions are more concise and easier to read than generator functions
- Generator expressions are suitable for simple scenarios

In [60]:
# List comprehension vs Generator expression
x = [number**2 for number in range(10)]
y = (number**2 for number in range(10))

## Use Cases

### A: Streaming Data

In [4]:
import time
import requests

def iss_position_stream():
    url = "http://api.open-notify.org/iss-now.json"
    while True:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            position = data['iss_position']
            latitude = position['latitude']
            longitude = position['longitude']
            yield (latitude, longitude)
        else:
            yield None
        time.sleep(2) 


In [ ]:
position = iss_position_stream()
for i, position in enumerate(position): 
    print(f"ISS Position: Latitude {position[0]}, Longitude {position[1]}")
    if i == 3:
        break

### B: Working with huge data

- **Pandas can read data in chunks.** 
- **These chunks behave much like generators:** They only load a chunk of data into memory at a time. We can use the `next()` function to load the next chunk. Or we can use a for loop to iterate over the chunks.
- They are useful when working with huge datasets that don't fit into memory.

In [19]:
import pandas as pd

First, we read in the entire data set about movies at once. Afterwards we just filter out movies from before 1910. If this were a huge data set, it would take a long time to load or might even not fit into memory.

In [ ]:
movies = pd.read_csv('movies.csv')
movies['year'] = pd.to_numeric(movies.title.str[-5:-1], errors='coerce')
movies[movies.year < 1910]

Now, we read in the data set in single chunks. Each chunk fits decently into memory. For each chunk we can filter out the relevant movies from before 1910, and the write the filtered data to a new file. This way we can work with huge data sets that don't fit into memory.

In [21]:
chunks = pd.read_csv('movies.csv', chunksize=10)      # The small chunk size is just for demonstration purposes
for chunk in chunks:
    chunk['year'] = pd.to_numeric(chunk.title.str[-5:-1], errors='coerce')
    chunk = chunk[chunk.year < 1910]
    chunk.to_csv('movies_before_1910.csv', mode='a', header=False)

### C: Infinite Iterators

In [23]:
def count():
    i = 0
    while True:
        yield i
        i += 1

In [ ]:
for number in count():
    print(number)
    if number > 10:
        break